In [1]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('miss_data').getOrCreate()

In [3]:
df = spark.read.csv("Python-and-Spark-for-Big-Data-master 2/Spark_DataFrames/ContainsNull.csv", inferSchema=True, header=True)

In [4]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [5]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [7]:
df.na.drop().show() # drop na wierszach, które mają jakąkolwiek w kolumnie wartość nieznaną

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [14]:
df.na.drop(thresh=2).show() # muszą być co najmniej dwie wartości inne niż NULL

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [16]:
df.na.drop(how='all').show() # how = 'all' usunie wiersze jeśli wszystkie kolumny są nieznane, basicowo jest to ustawione na how = 'any'

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [17]:
df.na.drop(subset=['Sales']).show() # usunie wszystkie wiersze, ktore nie maja w kolumnach wartosci na Sales

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [18]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [19]:
df.na.fill('FILL VALUE').show()

+----+----------+-----+
|  Id|      Name|Sales|
+----+----------+-----+
|emp1|      John| null|
|emp2|FILL VALUE| null|
|emp3|FILL VALUE|345.0|
|emp4|     Cindy|456.0|
+----+----------+-----+



In [20]:
df.na.fill(0).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [21]:
df.na.fill('No Name', subset=['Name']).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|No Name| null|
|emp3|No Name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



WYPEŁNIANIE DANYMI WIERSZY DFa ZA POMOCĄ ŚREDNIEJ ZE ZNANYCH WARTOŚCI

In [22]:
from pyspark.sql.functions import mean

In [24]:
mean_value = df.select(mean(df["Sales"])).collect()

In [28]:
mean_sales = mean_value[0][0]

In [33]:
df.na.fill(mean_sales, ["Sales"]).show() # nie potrzebujemy wpisywać nazwy parametru subset -> wystarczy, że podamy wartości w prawidłowej kolejności

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [35]:
df.na.fill(df.select(mean(df["Sales"])).collect()[0][0], ["Sales"]).show() # to samo co wyżej tylko w 1 linii

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

